In [30]:
%cd ..

/home/ltorres/leo/tesis/cloud-classification


In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from torchvision.io import read_image
import glob
import os
import math


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from random import sample
import torch
import torch.nn as nn
from torch.optim import lr_scheduler

In [33]:
from src.dataset import GCD
from src import config
from src import engine_gnn as engine
from src import utils

#from src.models.graph_nets import GraphConvGNN, GATConvGNN
from src.models.old.initial_graphnets import GATConvGNN

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
import wandb

In [36]:
wandb.login()

True

In [37]:
device = 'cuda:1'

In [38]:
LR = 2e-4
EPOCHS = 100

#### Data loaders

In [39]:
path_train_images = utils.get_gcd_paths(config.DATA_DIR,'train')

In [41]:
len(path_train_images)

10000

In [42]:
train_dataset = GCD(path_train_images, resize=256, aug_types='aug')

In [11]:
train_dataset = GCD(path_train_images, resize=256, aug_types='aug')

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=4,
        shuffle=True,
    )

In [12]:
path_test_images = utils.get_gcd_paths(config.DATA_DIR,'test')

test_dataset = GCD(path_test_images, resize=256)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=4,
        shuffle=False,
    )

### WANDB config file

In [13]:
exp_name = '6_GATConvGNN_LOGE_SGD_01_04_22'

In [14]:
wandb.init(
    # Set the project where this run will be logged
    project="cloud classification",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=exp_name, 
    # Track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "architecture": "GATConvGNN",
    "loss": "LogeLoss",
    "optim": "SGD",
    "dataset": "GCD",
    "epochs": config.EPOCHS,
  })

#### Model

In [15]:
from torchvision import models

In [16]:
torch.cuda.empty_cache()

In [ ]:
# model  = utils.build_model_gatconv(
#                                     7, #GCD num classes
#                                     512,
#                                     3,
#                                     4,
#                                     0.75,
#                                     device
#                                    )
model = GATConvGNN(7).to(device)

NameError: name 'GATConvGNN' is not defined

In [18]:
criterion = nn.CrossEntropyLoss()
#criterion = utils.loge_loss
#optimizer  = torch.optim.Adam(model.parameters(), lr=3e-4)
optimizer  = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)

scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [19]:
model_filename = f'/gcd_gnn/{exp_name}.pt'

In [20]:
class_mapper = {
    0: '1_cumulus',
    1: '2_altocumulus',
    2: '3_cirrus',
    3: '4_clearsky',
    4: '5_stratocumulus',
    5: '6_cumulonimbus',
    6: '7_mixed',
}

In [21]:
classes = list(class_mapper.values())

In [22]:
classes

['1_cumulus',
 '2_altocumulus',
 '3_cirrus',
 '4_clearsky',
 '5_stratocumulus',
 '6_cumulonimbus',
 '7_mixed']

### Train

In [ ]:
best_accuracy=0
best_iteration=0
best_params={}

for e in range(EPOCHS):
    ### TRAIN DATASET
    preds, targets, loss = engine.train_fn(model, train_loader, criterion, optimizer, device=device)
    train_acc = accuracy_score(targets, preds)
    
    scheduler.step()
    
    test_preds, test_targets, test_loss = engine.eval_fn(model, test_loader, criterion, device=device)
    test_acc = accuracy_score(test_targets, test_preds)
    
    if test_acc > best_accuracy:
        torch.save(model.state_dict(), config.SAVE_PATH+model_filename)
        print(f"Saved best parameters at epoch {e+1}")
        best_accuracy = test_acc
        best_iteration = e+1
        best_params = model.state_dict()
    
    print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format(
        e+1,
        train_acc,
        loss,
        test_acc,
        test_loss
    ))
    
    metrics = {
                "train/train_loss": loss,
                "train/train_accuracy": train_acc,
                "test/test_loss": test_loss,
                "test/test_accuracy": test_acc,
              }

    wandb.log(metrics)
    
# Load best model paramns    
model.load_state_dict(torch.load(config.SAVE_PATH+model_filename))

test_preds, test_targets, test_loss = engine.eval_fn(model, test_loader, criterion, device=device)
wandb.summary['test_accuracy'] = accuracy_score(test_targets, test_preds)
wandb.summary['test_loss'] = test_loss

wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            preds=test_preds, y_true=test_targets,
                            class_names=classes)})

wandb.finish()

100%|██████████| 282/282 [00:18<00:00, 14.92it/s]


Saved best parameters at epoch 1
EPOCH 1: Train acc: 70.28% Train Loss: 1.7579 Test acc: 73.42% Test Loss: 1.6797


100%|██████████| 282/282 [00:18<00:00, 15.44it/s]


Saved best parameters at epoch 2
EPOCH 2: Train acc: 84.87% Train Loss: 0.8457 Test acc: 76.84% Test Loss: 1.6523


100%|██████████| 282/282 [00:18<00:00, 15.18it/s]


...

100%|██████████| 282/282 [00:17<00:00, 15.76it/s]


EPOCH 79: Train acc: 94.04% Train Loss: 0.3226 Test acc: 78.31% Test Loss: 1.6977


100%|██████████| 282/282 [00:17<00:00, 15.74it/s]


EPOCH 80: Train acc: 94.33% Train Loss: 0.3110 Test acc: 78.60% Test Loss: 1.6568


 41%|████      | 115/282 [00:07<00:10, 16.33it/s]

In [ ]:
model.load_state_dict(torch.load(config.SAVE_PATH+model_filename))

In [ ]:
test_preds, test_targets, test_loss = engine.eval_fn(model, test_loader, criterion, device=device)

In [ ]:
accuracy_score(test_targets, test_preds)

---

### Accuracy per class and confusion matrix

In [ ]:
class_acc = dict()
matrix = np.zeros((7,7), dtype=int)

for i in range(7):
    pred_index = np.where(test_targets==i)[0]
    class_acc[class_mapper[i]] = accuracy_score(test_targets[pred_index], test_preds[pred_index])
    
    for j in test_preds[pred_index]:
        matrix[i,j]+=1 

In [ ]:
class_acc

In [ ]:
print(f"Global Test accuracy {accuracy_score(test_targets, test_preds)}")

sum_of_rows = matrix.sum(axis=1)
conf_mat = 100*matrix / sum_of_rows[:, np.newaxis]

df_cm = pd.DataFrame(conf_mat, index = class_acc.keys(),
                  columns = class_acc.keys())

plt.figure(figsize = (8,5))
plt.title("Confusion Matrix (Accuracy %)")
sns.heatmap(df_cm, annot=True)